# Analyis of grid hyperparameter search

In [ ]:
import json
import pathlib
import pandas as pd
import vaep.pandas

pd.options.display.max_columns = 30
pd.options.display.multi_sparse = False

In [ ]:
fname = "runs/experiment03/all_metrics.json"

In [ ]:
fname = pathlib.Path(fname)
FOLDER = fname.parent
with open(pathlib.Path(fname)) as f:
    metrics_dict = json.load(f)

In [ ]:
metrics = {}
for k, run_metrics in metrics_dict.items():
    metrics[k] = vaep.pandas.flatten_dict_of_dicts(run_metrics)

metrics = pd.DataFrame.from_dict(metrics, orient='index')
metrics

- options see [2Dline plot](https://matplotlib.org/stable/api/_as_gen/matplotlib.lines.Line2D.html#matplotlib.lines.Line2D)

In [ ]:
metrics["valid_pred_collab"]['intensity_predicted'].sort_values(
    'MSE').iloc[:10].plot(rot=90, 
                          x_compat=True, 
                          xticks=list(range(10)),
                          marker='o',
                          linestyle='',
                          title='Top 10 results for hyperparameters'
                         )

## Metadata

In [ ]:
from collections import namedtuple
from typing import List


class ExperimentTemplate():

    def __init__(self, template: str, prefix='experiment', sep='_'):
        self.template = template
        self.prefix = prefix
        self.sep = sep
        self.Entry = namedtuple('Entry', 'key name')
        self.keys = self.parse_template()
        self.map = {entry.key: entry.name for entry in self.keys}

    def parse_template(self) -> List[namedtuple]:
        #         ret = self.template[self.template.find('_')+1:]
        ret = self.template.split(f'{self.sep}', 1)[1]  # remove prefix
        ret = ret[:-1]
        ret = ret.split('}'+self.sep)
        ret = [self.Entry(*x.split(self.sep + '{')) for x in ret]
        return ret

    def get_meta_data(self, index: pd.Index) -> dict:
        meta = []
        for str_index in index:
            str_index = str_index.split(f'{self.sep}', 1)[1]
            str_index = str_index.split(self.sep)
            d = {str_index[i]:str_index[i+1] for i in range(0,len(str_index), 2)}
            d = {self.map[k]: v for k,v in d.items()}
            meta.append(d)
        return meta


name_template = "experiment_HL_{hidden_layers}_LD_{latent_dim}_E_{epochs}_BS_{batch_size}"

experiment_template = ExperimentTemplate(name_template)
meta_dict = experiment_template.get_meta_data(metrics.index)

In [ ]:
meta = pd.DataFrame(meta_dict, index=metrics.index)
meta

In [ ]:
metrics = metrics.set_index(pd.MultiIndex.from_frame(meta[['latent_dim', 'hidden_layers', 'batch_size']]))
metrics_styled = metrics.sort_index().style.background_gradient(
    # 'Greys',
    # 'BuPu',
    'viridis'
)
metrics_styled

In [ ]:
metrics_styled.to_excel(FOLDER/ 'metrics_styled.xlsx')

In [ ]:
metrics["valid_pred_collab"]['intensity_predicted'].sort_values(
    'MSE').iloc[:10].plot(rot=90,
                          x_compat=True,
                          xticks=list(range(10)),
                          marker='o',
                          linestyle=''
                          )